*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.4
IPython 6.2.1

torch 0.4.0


- Runs on CPU or GPU (if available)

# Model Zoo -- Multilayer Perceptron with BatchNorm

## Imports

In [1]:
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

## Settings and Dataset

In [2]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [3]:
numbatches = len(train_loader)

In [4]:
##########################
### MODEL
##########################

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        # The following to lones are not necessary, 
        # but used here to demonstrate how to access the weights
        # and use a different weight initialization.
        # By default, PyTorch uses Xavier/Glorot initialization, which
        # should usually be preferred.
        self.linear_1.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()
        self.linear_1_bn = torch.nn.BatchNorm1d(num_hidden_1)
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2.weight.detach().normal_(0.0, 0.1)
        self.linear_2.bias.detach().zero_()
        self.linear_2_bn = torch.nn.BatchNorm1d(num_hidden_2)
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.detach().normal_(0.0, 0.1)
        self.linear_out.bias.detach().zero_()
        
    def forward(self, x):
        out = self.linear_1(x)
        # note that batchnorm is in the classic
        # sense placed before the activation
        out = self.linear_1_bn(out)
        out = F.relu(out)
        
        out = self.linear_2(out)
        out = self.linear_2_bn(out)
        out = F.relu(out)
        
        logits = self.linear_out(out)
        probas = F.log_softmax(logits, dim=1)
        return  logits,probas

    
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

model = model.to(device)

##########################
### COST AND OPTIMIZER
##########################

cost_fn = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits, probas = model(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100
    

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = cost_fn(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     numbatches, cost))

    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/937 | Cost: 2.6465
Epoch: 001/010 | Batch 050/937 | Cost: 1.0305
Epoch: 001/010 | Batch 100/937 | Cost: 0.5404
Epoch: 001/010 | Batch 150/937 | Cost: 0.4430
Epoch: 001/010 | Batch 200/937 | Cost: 0.3235
Epoch: 001/010 | Batch 250/937 | Cost: 0.1927
Epoch: 001/010 | Batch 300/937 | Cost: 0.4007
Epoch: 001/010 | Batch 350/937 | Cost: 0.3802
Epoch: 001/010 | Batch 400/937 | Cost: 0.2528
Epoch: 001/010 | Batch 450/937 | Cost: 0.2257
Epoch: 001/010 | Batch 500/937 | Cost: 0.1454
Epoch: 001/010 | Batch 550/937 | Cost: 0.2160
Epoch: 001/010 | Batch 600/937 | Cost: 0.3425
Epoch: 001/010 | Batch 650/937 | Cost: 0.2176
Epoch: 001/010 | Batch 700/937 | Cost: 0.2305
Epoch: 001/010 | Batch 750/937 | Cost: 0.3723
Epoch: 001/010 | Batch 800/937 | Cost: 0.2464
Epoch: 001/010 | Batch 850/937 | Cost: 0.1278
Epoch: 001/010 | Batch 900/937 | Cost: 0.1296
Epoch: 001/010 training accuracy: 95.64%
Epoch: 002/010 | Batch 000/937 | Cost: 0.2131
Epoch: 002/010 | Batch 050/937 | Cost: 

Epoch: 009/010 | Batch 900/937 | Cost: 0.0150
Epoch: 009/010 training accuracy: 99.43%
Epoch: 010/010 | Batch 000/937 | Cost: 0.0170
Epoch: 010/010 | Batch 050/937 | Cost: 0.0063
Epoch: 010/010 | Batch 100/937 | Cost: 0.0249
Epoch: 010/010 | Batch 150/937 | Cost: 0.0056
Epoch: 010/010 | Batch 200/937 | Cost: 0.0827
Epoch: 010/010 | Batch 250/937 | Cost: 0.0170
Epoch: 010/010 | Batch 300/937 | Cost: 0.0541
Epoch: 010/010 | Batch 350/937 | Cost: 0.0285
Epoch: 010/010 | Batch 400/937 | Cost: 0.0692
Epoch: 010/010 | Batch 450/937 | Cost: 0.0054
Epoch: 010/010 | Batch 500/937 | Cost: 0.0544
Epoch: 010/010 | Batch 550/937 | Cost: 0.0228
Epoch: 010/010 | Batch 600/937 | Cost: 0.0322
Epoch: 010/010 | Batch 650/937 | Cost: 0.0111
Epoch: 010/010 | Batch 700/937 | Cost: 0.0261
Epoch: 010/010 | Batch 750/937 | Cost: 0.0788
Epoch: 010/010 | Batch 800/937 | Cost: 0.0216
Epoch: 010/010 | Batch 850/937 | Cost: 0.0676
Epoch: 010/010 | Batch 900/937 | Cost: 0.0499
Epoch: 010/010 training accuracy: 99.62

In [6]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.93%


In [7]:
%load_ext watermark
%watermark -iv

numpy     1.15.4
torch     1.0.0
torchvision0.2.1

